In [193]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
import sys

# Set the maximum width for text output
pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display

# Surprise!

People's decisions depend on the way probability is expressed to them. A lot of financial models assume Normal / Gaussian distribution  due to its mathematical tractability. However, as a thin-tailed model, it underestimates the likelihood of extreme events when compared to many real-world phenomena. On the other hand, other distributions, like Student's t or Pareto, can account for extreme values with their fat / heavy tails, which is relevant in the context of modelling the occurrence of outlier events. This property makes it more appropriate for financial returns data (especially at higher frequencies like daily returns), where it's important to estimate the likelihood of very large losses. The difference between the Normal and the other mentioned distributions can be quantified as "unexpectedness", or "surprise. 

In [219]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.stats import norm, t

# # Download historical data for Tesla stock
# data = yf.download('TSLA') #,'2018-01-01','2023-07-21'datetime. now()  

# # Remove NA values
data = data.dropna()

In [229]:
# # Calculate daily returns
data['Return'] = data['Close'].pct_change()

# Calculate the cumulative returns
df['Cumulative Returns'] = (1 + df['Return']).cumprod()

# Calculate the standard deviation and mean
sigma = data['Return'].std()
mu = data['Return'].mean()

# Calculate MAD
mad = np.mean(np.abs(data['Return'] - mu))
# mad = np.nanmedian(np.abs(data['Return'] - np.nanmedian(data['Return'])))

# Calculate the number of standard deviations each return is from the mean
data['Return Z-Scores'] = (data['Return'] - mu) / sigma

# Count the number of returns beyond 3 standard deviations
num_beyond_3sd = len(data[np.abs(data['Return Z-Scores']) > 3])

# Get the total number of returns
total_num = len(data['Return'])

# Calculate the percentage of returns beyond 3 standard deviations
percentage_beyond_3sd = num_beyond_3sd / total_num * 100

# print(f'Tesla returns: {percentage_beyond_3sd:.2f}% of data is beyond 3 standard deviations')

# Calculate Z-scores
z_scores = np.arange(-3, 4, 1)
x_z_scores = mu + z_scores * sigma

In [230]:
# Generate normal distribution with the mean and standard deviation
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
# x = np.linspace(mu_posterior_mean - 3*sd_posterior_mean, mu_posterior_mean + 3*sd_posterior_mean, 100)
x_minmax = np.linspace(data['Return'].min(), data['Return'].max(), 100)

# pdf_norm = norm.pdf(x, mu, sigma)
pdf_norm = norm.pdf(x_minmax, mu, sigma)
# pdf_norm = norm.pdf(x_minmax, mu_posterior_mean, sd_posterior_mean)

from scipy.stats import t

# Estimate parameters for t-distribution
# v, loc, scale = t.fit(data['Return'])
# v = 3  # degrees of freedom
# rv = t(v)

# Generate t-distribution
# x_t = np.linspace(t.ppf(0.01, df, loc, scale), t.ppf(0.99, df, loc, scale), 100)
pdf_t = t.pdf(x_minmax, v, loc, scale)
# pdf_t = rv.pdf((x-mu)/sigma) / sigma

In [231]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Create histogram (this will be displayed on the left y-axis)
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Return'], histnorm='probability', nbinsx=50, yaxis='y1',
                           name='Histogram of Returns', marker=dict(color='rgb(155,201,251)')))

# Create normal distribution (this will be displayed on the right y-axis)
fig.add_trace(go.Scatter(x=x_minmax, y=pdf_norm, mode='lines', name='Normal Distribution', 
                         yaxis='y2', marker=dict(color='rgb(255,142,195)')))
fig.add_trace(go.Scatter(x=x_minmax, y=pdf_t, mode='lines', name='T-Distribution', 
                         yaxis='y2', marker=dict(color='rgb(77,199,125)')))

# #Z-SCORES
# Create subplots: use 'domain' type for secondary y-axis
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add Z-score lines to the plot
# for z, x in zip(z_scores, x_z_scores):
#     fig.add_trace(go.Scatter(x=[x, x], y=[0, 0.2], mode='lines', name=f'Z={z}'))
#--------

# Add a scatter trace for the shape object to show up in the legend
# No values, so the line doesn't actually show on the plot
fig.add_trace(go.Scatter(x=[None], y=[None], 
    mode='lines', line=dict(color='darkgrey', dash='dot'), name='SD'
))

# Add a scatter trace for the shape object 2 to show up in the legend
fig.add_trace(go.Scatter(x=[None], y=[None], 
    mode='lines', line=dict(color='darkgrey', dash='dash'), name='MAD'
))

# # Add SD and MAD lines
fig.add_shape(type="line", x0=mu+sigma, y0=0, x1=mu+sigma, y1=0.2, name='+1 SD', 
              line=dict(color="darkgrey",width=1, dash="dot"))
fig.add_shape(type="line", x0=mu-sigma, y0=0, x1=mu-sigma, y1=0.2, name='-1 SD',
              line=dict(color="darkgrey",width=1, dash="dot"))
fig.add_shape(type="line", x0=mu+sigma*2, y0=0, x1=mu+sigma*2, y1=0.2, name='+1 SD', 
              line=dict(color="darkgrey",width=1, dash="dot"))
fig.add_shape(type="line", x0=mu-sigma*2, y0=0, x1=mu-sigma*2, y1=0.2, name='-1 SD',
              line=dict(color="darkgrey",width=1, dash="dot"))
fig.add_shape(type="line", x0=mu+sigma*3, y0=0, x1=mu+sigma*3, y1=0.2, name='+1 SD', 
              line=dict(color="darkgrey",width=1, dash="dot"))
fig.add_shape(type="line", x0=mu-sigma*3, y0=0, x1=mu-sigma*3, y1=0.2, name='-1 SD',
              line=dict(color="darkgrey",width=1, dash="dot"))

fig.add_shape(type="line", x0=mu+mad, y0=0, x1=mu+mad, y1=0.2, name='+1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))
fig.add_shape(type="line", x0=mu-mad, y0=0, x1=mu-mad, y1=0.2, name='-1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))
fig.add_shape(type="line", x0=mu+mad*2, y0=0, x1=mu+mad*2, y1=0.2, name='+1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))
fig.add_shape(type="line", x0=mu-mad*2, y0=0, x1=mu-mad*2, y1=0.2, name='-1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))
fig.add_shape(type="line", x0=mu+mad*3, y0=0, x1=mu+mad*3, y1=0.2, name='+1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))
fig.add_shape(type="line", x0=mu-mad*3, y0=0, x1=mu-mad*3, y1=0.2, name='-1 MAD',
              line=dict(color="darkgrey",width=1, dash="dash"))

# Add "caption"
fig.add_annotation(
    xref="paper", yref="paper",  # 'paper' coordinates mean the location will be relative to the size of the figure
    x=0.02, y=-0.125,  # position of the caption (0.5, -0.2) in 'paper' coordinates
    text="Fig.1",  # your caption text
    showarrow=False, font=dict(size=12, color="black"),
    align="left",  # align the caption to the center
    borderwidth=1, borderpad=4, #bordercolor="black",
    bgcolor="white",  # background color of the caption
    opacity=0.7  # opacity of the caption
)

# Update layout
fig.update_layout(
#     title_text="Distribution of Tesla Returns<br><sub>        The histogram is 'density' normalized, the area under the histogram sums up to 1. This allows the histogram to be interpreted in the same way as a PDF.<br></sub><sub>        Returns are (Price_today - Price_yesterday) / Price_yesterday.</sub><br>", 
    title={
    'text': "Distribution of Tesla Returns",
        #<br><sub>he histogram is 'density' normalized, the area under the histogram sums up to 1. This allows the histogram to be interpreted in the same way as a PDF.<br><br></sub>",
    'y':0.95, 'x':0.45, 'xanchor': 'center', 'yanchor': 'top'},
    xaxis_title_text='Return',
    yaxis=dict(title_text='Density (Histogram)', side='left',
               #titlefont=dict(color='blue'), tickfont=dict(color='blue')
    ),
    yaxis2=dict(title_text='Density (PDF)', side='right', overlaying='y', range=[0, 16]
    ),
    bargap=0.2, bargroupgap=0.1, autosize=False, width=1000, height=800,
)

fig.show()

In [ ]:
The standard deviation (std) and mean absolute deviation (MAD) are two different measures of variability in a data set, and they give different weightings to deviations from the mean.

Standard deviation squares the differences from the mean before averaging them. This gives more weight to larger differences. It's a measure of how spread out numbers in the data are, and it's more influenced by outliers or extreme values because of the squaring operation. So when you calculate mu + 3*std, you're finding a value that's three standard deviations away from the mean, a region that, in a normal distribution, encompasses about 99.7% of the data.

On the other hand, MAD is the average of the absolute differences from the mean. It's a simpler calculation that gives equal weight to all values, and it's less influenced by extreme values. So when you calculate mu + 3*MAD, you're finding a value that's three mean absolute deviations away from the mean, a region that encompasses a certain percentage of the data that's not as easily defined as it is with standard deviations in a normal distribution.

Standard deviation is a more conservative measure of dispersion for data with outliers, since it tends to give a larger spread. However, MAD can give a better idea of the "typical" deviation, since it's not as influenced by extreme values.

In [171]:
print(f'Tesla returns: {percentage_beyond_3sd:.2f}% of data is beyond 3 standard deviations')

Tesla returns: 1.64% of data is beyond 3 standard deviations


The probability density function (PDF) lines represent the probability density of the returns at each point. The y-axis (right side) represents the density values for these probability distributions. It's important to note that these densities are not direct probabilities but instead give a measure of how much the data is concentrated around a particular value.

A density can take a value greater than 1 because it's not a probability; it's a rate per unit (in this case, per unit return). The height of the PDF at any given point is proportional to the probability density of the returns at that point, assuming the particular distribution.

The fact that some of the values go up to 16 is not an issue. It merely reflects the fact that the data is more concentrated around certain values. The precise values of the density are not often of primary interest; usually, we care more about the overall shape of the distribution.

If you integrate (i.e., find the area under) the PDF across all possible values (from negative infinity to positive infinity), you should get 1. This is because the total probability of all possible outcomes (all possible returns, in this case) should sum up to 1.

In the context of the returns of the Tesla stock, the PDFs provide a way to visualize how the returns are distributed according to different statistical models (normal, t-distribution, etc.). By comparing the shape of these PDFs with the histogram of the actual returns, you can assess the goodness-of-fit of these models.
_________

Let's try to quantify how much a particular price movement deviates from what we would expect under normal circumstances. Assuming TSLA's daily returns follow a fat-tailed distribution, most days will see relatively small price changes, but occasionally, there will be very large changes -- these are the "fat tails".

One way to quantify the "surprise" of a price movement is by calculating how many standard deviations away from the mean it is (or "z-score"). Under a normal distribution, a z-score of 3 or more is considered very rare (occurring less than 0.3% of the time), but under a fat-tailed distribution, such occurrences are more common.

Let's say, for example, the average daily return of TSLA is 0.002% with a standard deviation of 0.036%. If TSLA one day has a return of 0.24%, we can calculate the z-score as follows:

$z = (0.24\% - 0.002\%) / 0.036\% = 6.7$

This suggests the move is 6.7 standard deviations away from the mean, a huge "surprise" under normal distribution assumptions, but in a fat-tailed world, such moves can happen more often than we would normally expect.

In [198]:
# (data['Return'].max() - mu) / sigma

In [ ]:
The Student's t-distribution is a family of probability distributions that arises when estimating the mean of a normally distributed population in situations where the sample size is small and the population standard deviation is unknown. It has a parameter called degrees of freedom, typically denoted by ν (nu) or df, which is related to the sample size. It refers to the number of independent pieces of information available to estimate parameters of a population from a sample. 

For a simple sample from a normal distribution, the degrees of freedom would be the sample size minus 1 (i.e., if you have a sample of size n, then df = n - 1). This is because you've used up one "degree of freedom" to calculate the sample mean.

Degrees of freedom are crucial to the shape of the t-distribution. As the degrees of freedom increase, the t-distribution becomes more like a standard normal distribution. When you have a large sample size (and thus many degrees of freedom), the extra uncertainty from estimating the population standard deviation becomes negligible, and the t-distribution converges to a normal distribution. But with fewer degrees of freedom (smaller sample size), the t-distribution has heavier tails than the normal distribution, reflecting the greater uncertainty.

Raphael: Whatever hedging, insuring, protection, risk management policy, do not rely on a model you have selected, then estimated, but perform thorough pseudo-random (Monte-Carlo) simulations that includes these fluctuations and model changes. Now comes the interesting part: start with a thin-tailed model, say, result of CLT (central limit theorem), Brownian, Gaussian... Then make parameters fluctuate and model change with some random occurrence. Then repeat the process: make fluctuation parameters themselves fluctuate, and the model change itself change, etc. What you get in the end is exactly a Pareto distribution. The tail parameter of which depends on size of repeated fluctuations and model changes. But the bottom line is the systematicity of reaching a Pareto distribution and a fractal structure. This is why, in real life, we always observe fractal structures, self similarity and Pareto distribution. This is an extremely robust pattern.

In [ ]:
A "fat tail" in statistics refers to the tails of a probability distribution that decay more slowly than an exponential decay, which is the case in normal (or Gaussian) distributions. In other words,  extreme events (those far from the mean or median) are more likely in a fat-tailed distribution than in a thin-tailed one.

A power law distribution is a specific type of fat-tailed distribution. It follows the form:

$P(x)\~x^(-alpha)$

where P(x) is the probability of an event of size x, alpha is a constant (often > 1), and '~' indicates 'is proportional to.' An important feature of power law distributions is that they are scale invariant, meaning that the shape of the distribution remains the same, irrespective of the scale at which it is viewed.

In the context of finance this is important because fat-tailed distributions and power laws often more accurately describe real-world phenomena (like fluctuations in stock prices) than thin-tailed distributions like the normal distribution. They capture the higher likelihood of extreme events or "black swans," which can have major impacts on markets and economies.

### Further exploration

Surprise can be measured as **the difference between the earnings expectations (Gaussian), and actual earnings (nonlinear**, based on a variety of factors including changes in market conditions, the company's strategic decisions, and unforeseen events). 

Analysts' expectations are usually reflected in the form of earnings estimates, which are frequently collected and reported by a number of financial information and news services. These include:

- Earnings Forecasts: These are typically expressed as an estimate of Earnings Per Share (EPS) for a given quarter or year. The EPS estimate represents the analysts' average expectation of a company's profitability.

- Revenue Estimates: Analysts also provide estimates for a company's revenue for a given period. This helps investors understand the expected top-line growth of the company.

- Other Financial Metrics: Depending on the company and industry, analysts may also provide estimates for other key financial metrics, such as gross margin, operating margin, EBITDA, and more.


We could look into approximating the concept of "priced in" - the extent to which market expectations, including all publicly available information, are already reflected in the current price of a security. Some of the factors that might be "priced in":

- Publicly Available Information: This includes financial reports, news releases, and any other information that's publicly available and pertinent to the value of the asset.

- Market Sentiment: This is the overall attitude of investors towards a particular security or financial market. It is the cumulative attitude of all market participants towards risk, and it can heavily influence whether potential news or events are already priced in.

- Analyst Estimates: If analysts have widely shared an expectation for a company’s future performance, this expectation may be priced in.

- Economic Indicators: Things like GDP reports, unemployment rates, and consumer sentiment indices can all impact what's priced into the market.

In [45]:
df=data.copy()

In [65]:
# # Filter the last 5 years
start_date = df.index.max() - pd.DateOffset(years=5)
df_last_5_years = df.loc[start_date:]

In [235]:
import pandas as pd
import plotly.graph_objs as go
from IPython.display import display

# Create a Scatter plot for the returns
trace = go.Scatter(
    x=df_last_5_years.index, 
    y=df_last_5_years['Close'], 
    mode='lines',
    name='Returns'
)

# Create the list of events
events = [
    {'date': '2018-08-06', 'event': "Musk's 'funding secured' tweet"},
    {'date': '2019-11-21', 'event': 'Cybertruck Reveal'},
    {'date': '2020-07-22', 'event': 'Q2 2020 Earnings Report'},
    {'date': '2020-08-31', 'event': '5-for-1 stock split'},
    {'date': '2020-12-21', 'event': 'Inclusion in the S&P 500'},
    {'date': '2023-07-19', 'event': "23' Q2 Earnings"},
    {'date': '2023-04-19', 'event': "23' Q1 Earnings"},
    {'date': '2023-01-25', 'event': "22' Q4 Earnings"},
    {'date': '2022-10-19', 'event': "22' Q3 Earnings"},
    {'date': '2022-07-20', 'event': "22' Q2 Earnings"},
    {'date': '2022-04-20', 'event': "22' Q1 Earnings"},
    {'date': '2022-01-26', 'event': "21' Q4 Earnings"},
    {'date': '2021-10-20', 'event': "21' Q3 Earnings"},
    {'date': '2021-07-26', 'event': "21' Q2 Earnings"},
    {'date': '2021-04-21', 'event': "21' Q1 Earnings"},   
]

# Add the annotations
annotations = []
for event in events:
    annotations.append(
        go.layout.Annotation(
            x=event['date'],
            y=df.loc[event['date'], 'Close'], # The return on the event date
            xref="x", yref="y",
            text=event['event'],
            showarrow=True,
            arrowhead=1,
#             height=1,
            ax=-10, ay=-30
        )
    )

# Create the layout
layout = go.Layout(
    title='Tesla 5-Year Price with Key Events', 
    annotations=annotations,
    yaxis=dict(title='Price'),
)

# Create the figure and add the scatter plot
fig = go.Figure(data=[trace], layout=layout)

# # Display the graph using IPython's display() function
# display(fig)

# Display the figure
fig.show()